In [2]:
import sys
sys.path.append('../../')
import time
import argparse
import os
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

##### 导入数据

In [5]:
root_path = "../../data/"
output = pd.read_table(f"./output/ERM_E_1.txt",header=None)
label = pd.read_table('../../data/ERM/label.dat.test',header=None)
label = label.sort_values(0)
output.columns = ['Idx','Ept','GroudTruth','Predict']
output['GroudTruth'] = label[3].values
output = output[['Idx','GroudTruth','Predict']]
test_node = pd.read_csv("../../data/Process/test_node.csv")
train_node = pd.read_csv("../../data/Process/train_node.csv")

In [6]:
target = output[(output['GroudTruth']!=1) & (output['Predict']==1)]
out = test_node[test_node['nid'].isin(target.Idx.values)]
out

,id,amino_acid_seq:String,~label,label,nid
829,b0655,MQLRKPATAILALALSAGLAQADDAAPAAGSTLDKIAKNGVIVVGH...,0,Gene,1698


In [7]:
test_fasta_df = test_node[["id","amino_acid_seq:String"]]
train_fasta_df = train_node[["id","amino_acid_seq:String"]]
test_fasta_df

,id,amino_acid_seq:String
0,b3064,MRVLGIETSCDETGIAIYDDEKGLLANQLYSQVKLHADYGGVVPEL...
1,b3594,MSNTYQKRKASKEYGLYNQCKKLNDDELFRLLDDHNSLKRISSARV...
2,b2918,MINEATLAESIRRLRQGERATLAQAMTLVESRHPRHQALSTQLLDA...
3,b4015,MKTRTQQIEELQKEWTQPRWEGITRPYSAEDVVKLRGSVNPECTLA...
4,b3176,MSNRKYFGTDGIRGRVGDAPITPDFVLKLGWAAGKVLARHGSRKII...
...,...,...
905,b4396,MDQAGIIRDLLIWLEGHLDQPLSLDNVAAKAGYSKWHLQRMFKDVT...
906,b3943,MSAAGKSNPLAISGLVVLTLIWSYSWIFMKQVTSYIGAFDFTALRC...
907,b1204,MTIKLIVGLANPGAEYAATRHNAGAWFVDLLAERLRAPLREEAKFF...
908,b2224,MKNCVIVSAVRTAIGSFNGSLASTSAIDLGATVIKAAIERAKIDSQ...


##### fasta文件

In [8]:
# 指定保存FASTA文件的路径
fasta_file_path = 'output/test.fasta'

# 打开文件并将DataFrame写入FASTA文件
with open(fasta_file_path, 'w') as fasta_file:
    for index, row in test_fasta_df.iterrows():
        header = '>' + row['id'] + '\n'
        sequence = row['amino_acid_seq:String'] + '\n'
        fasta_file.write(header)
        fasta_file.write(sequence)

print(f'FASTA文件已保存到: {fasta_file_path}')

FASTA文件已保存到: output/test.fasta


In [9]:
# 指定保存FASTA文件的路径
fasta_file_path = 'output/train.fasta'

# 打开文件并将DataFrame写入FASTA文件
with open(fasta_file_path, 'w') as fasta_file:
    for index, row in train_fasta_df.iterrows():
        header = '>' + row['id'] + '\n'
        sequence = row['amino_acid_seq:String'] + '\n'
        fasta_file.write(header)
        fasta_file.write(sequence)

print(f'FASTA文件已保存到: {fasta_file_path}')

FASTA文件已保存到: output/train.fasta


##### DeepTFPredict

In [ ]:
!cd /home/linjw/BioML/deeptfactor 
!python tf_running.py -i output/test.fasta -o output/ -g cpu

In [13]:
deeptfpredcit = pd.read_table(f"./output/prediction_result.txt")
deeptfpredcit

,sequence_ID,prediction,score
0,b0023,False,0.0045
1,b1783,False,0.0096
2,b0192,False,0.0075
3,b1386,False,0.0074
4,b0463,False,0.0046
...,...,...,...
895,b1501,False,0.2175
896,b1328,True,0.9973
897,b4216,False,0.0099
898,b3811,False,0.0089


In [14]:
dp_pd = pd.merge(test_node,deeptfpredcit,left_on='id', right_on='sequence_ID')
dp_pd

,id,amino_acid_seq:String,~label,label,nid,sequence_ID,prediction,score
0,b0068,MLKKCLPLLLLCTAPVFAKPVLTVYTYDSFAADWGPGPVVKKAFEA...,0,Gene,2070,b0068,False,0.0078
1,b2405,MERVYRTDLKLLRYFLAVAEELHFGRAAARLNMSQPPLSIHIKELE...,1,Gene,558,b2405,True,0.9977
2,b3651,MNPTRYARICEMLARRQPDLTVCMEQVHKPHNVSAIIRTADAVGVH...,0,Gene,1231,b3651,False,0.0200
3,b0773,MKLISNDLRDGDKLPHRHVFNGMGYDGDNISPHLAWDDVPAGTKSF...,0,Gene,1537,b0773,False,0.0091
4,b0860,MKKLVLAALLASFTFGASAAEKINFGVSATYPPFESIGANNEIVGF...,0,Gene,1837,b0860,False,0.0067
...,...,...,...,...,...,...,...,...
279,b3848,MESWLIPAAPVTVVEEIKKSRFITMLAHTDGVEAAKAFVESVRAEH...,0,Gene,1334,b3848,False,0.0161
280,b3671,MASSGTTSTRKRFTGAEFIVHFLEQQGIKIVTGIPGGSILPVYDAL...,0,Gene,1457,b3671,False,0.0102
281,b2418,MSSLLLFNDKSRALQADIVAVQSQVVYGSVGNSIAVPAIKQNGLNV...,0,Gene,544,b2418,False,0.0085
282,b1204,MTIKLIVGLANPGAEYAATRHNAGAWFVDLLAERLRAPLREEAKFF...,0,Gene,2834,b1204,False,0.0113


In [15]:
dp_report = pd.DataFrame(classification_report(dp_pd["~label"].values, dp_pd["prediction"].values, zero_division=0, output_dict=True))
dp_report

,0,1,accuracy,macro avg,weighted avg
precision,1.000000,0.850000,0.989437,0.925000,0.991021
recall,0.988764,1.000000,0.989437,0.994382,0.989437
f1-score,0.994350,0.918919,0.989437,0.956635,0.989835
support,267.000000,17.000000,0.989437,284.000000,284.000000
